# Обучение нейронной сети для решения задачи NER

In [3]:
import pandas as pd
from tqdm import tqdm
import json
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel, AutoModelForTokenClassification
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import ExponentialLR
from sklearn.metrics import accuracy_score, f1_score

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
with open('/content/drive/MyDrive/data4_allnames.json', 'r') as json_file:
    data = json.load(json_file)['data']

In [6]:
tokenizer = AutoTokenizer.from_pretrained('DeepPavlov/xlm-roberta-large-en-ru')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/582 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/944k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.51M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

In [7]:
class dataset(Dataset):
    def __init__(self, data: dict, tokenizer, max_len: int):
        self.len = len(data)
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len
        print('Label prepare')
        self.__prepare_labels__()

    def __prepare_labels__(self) -> list:
        labels = []
        tokens_l = []
        for i in tqdm(range(0, len(self.data))):
            text = self.data[i]['text']
            name = self.data[i]['name']
            name.sort()
            startidx = 0
            IO = []
            tokens_t = []
            for j in range(len(name)):
                tokens = self.tokenizer.tokenize(text[startidx:name[j][0]])
                IO.extend([0]*len(tokens))
                tokens_t.extend(tokens)

                tokens = self.tokenizer.tokenize(text[name[j][0]:name[j][1]])
                IO.extend([1]*len(tokens))
                tokens_t.extend(tokens)
                startidx = name[j][1]
            tokens = self.tokenizer.tokenize(text[startidx:])
            IO.extend([0]*len(tokens))
            tokens_t.extend(tokens)

            labels.append(IO)
            tokens_l.append(tokens_t)
        self.labels = labels
        self.tokens = tokens_l

    def __getitem__(self, idx):
        tokenized_text = self.tokens[idx]
        labels = self.labels[idx]
        maxlen = self.max_len

        if (len(tokenized_text) > maxlen):
          tokenized_text = tokenized_text[:maxlen]
          labels = labels[:maxlen]
        else:
          tokenized_text = tokenized_text + ['[PAD]' for _ in range(maxlen - len(tokenized_text))]
          labels = labels + [0 for _ in range(maxlen - len(labels))]

        attn_mask = [1 if tok != '[PAD]' else 0 for tok in tokenized_text]

        ids = self.tokenizer.convert_tokens_to_ids(tokenized_text)

        return {
              'ids': torch.tensor(ids, dtype=torch.long),
              'mask': torch.tensor(attn_mask, dtype=torch.long),
              'targets': torch.tensor(labels, dtype=torch.long)
        }

    def __len__(self):
        return self.len

In [8]:
train_data, val_data = train_test_split(data,
                                        random_state = 77,
                                        test_size = 0.3)

In [9]:
MAX_LEN = 512
train_dataset = dataset(train_data, tokenizer, MAX_LEN)
val_dataset = dataset(val_data, tokenizer, MAX_LEN)

Label prepare


100%|██████████| 11091/11091 [00:16<00:00, 666.62it/s]


Label prepare


100%|██████████| 4754/4754 [00:08<00:00, 589.25it/s]


In [12]:
train_params = {'batch_size': 3,
                }

test_params = {'batch_size': 3,
                }

train_loader = DataLoader(train_dataset, shuffle=True, **train_params)
val_loader = DataLoader(val_dataset, **test_params)

In [13]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [14]:
model = AutoModelForTokenClassification.from_pretrained('DeepPavlov/xlm-roberta-large-en-ru',
                                                   num_labels=2)
model.to(device)

config.json:   0%|          | 0.00/722 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.36G [00:00<?, ?B/s]

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at DeepPavlov/xlm-roberta-large-en-ru and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


XLMRobertaForTokenClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(35054, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=1024, out_featur

In [15]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.00001)
scheduler = ExponentialLR(optimizer, gamma=0.9)

In [16]:
def train():
    tr_accuracy = []
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_loss = []
    model.train()
    acc = []
    f1 = []

    for idx, batch in tqdm(enumerate(train_loader)):

        ids = batch['ids'].to(device, dtype = torch.long)
        mask = batch['mask'].to(device, dtype = torch.long)
        targets = batch['targets'].to(device, dtype = torch.long)

        outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
        loss, tr_logits = outputs.loss, outputs.logits
        tr_loss.append(loss.item())

        nb_tr_steps += 1

        # compute training accuracy
        flattened_targets = targets.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        # now, use mask to determine where we should compare predictions with targets
        active_accuracy = mask.view(-1) == 1 # active accuracy is also of shape (batch_size * seq_len,)
        targets = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)

        acc.append(accuracy_score(targets.to('cpu').numpy(), predictions.to('cpu').numpy()))
        f1.append(f1_score(targets.to('cpu').numpy(), predictions.to('cpu').numpy()))

        if len(tr_loss) % 50 == 0:
            print("Training loss: ", sum(tr_loss[-50:]) / len(tr_loss[-50:]))
            print("Training accuracy: ", sum(acc[-50]) / len(acc[-50:]))
            print("Training F1: ", sum(f1[-50:]) / len(f1[-50:]))

        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    scheduler.step()

    epoch_loss = sum(tr_loss[-32:]) / len(tr_loss[-32:])
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training Accuracy epoch: {acc_sum/acc_amount}")

In [17]:
train()

50it [00:48,  1.04it/s]

Training loss:  0.08912918768823147
Training accuracy:  0.9091251702725777
Training F1:  0.008


99it [01:36,  1.01it/s]

Training loss:  0.03790824256837368
Training accuracy:  0.9611403279706139
Training F1:  0.6606414434830158


149it [02:27,  1.01s/it]

...

3649it [1:01:28,  1.01s/it]

Training loss:  0.0027727286165463738
Training accuracy:  0.9977795339511398
Training F1:  0.9863891905571383


3697it [1:02:17,  1.01s/it]

Training loss epoch: 0.002479028777770509


NameError: name 'acc_sum' is not defined

In [18]:
torch.save(model.state_dict(), '/content/drive/MyDrive/model_xlm_9864.pickle')

In [19]:
def val():
    tr_accuracy = []
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_loss = []
    model.eval()
    acc = []
    f1 = []

    for idx, batch in tqdm(enumerate(val_loader)):

        ids = batch['ids'].to(device, dtype = torch.long)
        mask = batch['mask'].to(device, dtype = torch.long)
        targets = batch['targets'].to(device, dtype = torch.long)


        outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
        loss, tr_logits = outputs.loss, outputs.logits
        tr_loss.append(loss.item())

        nb_tr_steps += 1

        flattened_targets = targets.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        # now, use mask to determine where we should compare predictions with targets
        active_accuracy = mask.view(-1) == 1 # active accuracy is also of shape (batch_size * seq_len,)
        targets = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)

        acc.append(accuracy_score(targets.to('cpu').numpy(), predictions.to('cpu').numpy()))
        f1.append(f1_score(targets.to('cpu').numpy(), predictions.to('cpu').numpy(), average="macro"))

        if len(tr_loss) % 25 == 0:
            print("Val loss: ", sum(tr_loss[-25:]) / len(tr_loss[-25:]))
            print("Val accuracy: ", sum(acc[-25:]) / len(acc[-25:]))
            print("Val F1: ", sum(f1[-25:]) / len(f1[-25:]))

    epoch_loss = sum(tr_loss[-32:]) / len(tr_loss[-32:])
    print(f"Val loss epoch: {epoch_loss}")

In [21]:
val()

25it [00:07,  3.17it/s]

Val loss:  0.001369015403688536
Val accuracy:  0.9988904325579459
Val F1:  0.9962766109320836


50it [00:15,  3.21it/s]

Val loss:  0.0012710119283292443
Val accuracy:  0.9992749782370385
Val F1:  0.9978500800021165


75it [00:23,  3.19it/s]

...

Val loss:  0.0015839362939004786
Val accuracy:  0.9993699651087361
Val F1:  0.9971142282432217


1575it [08:37,  3.02it/s]

Val loss:  0.001382182610250311
Val accuracy:  0.9983589040811826
Val F1:  0.9940853994693647


1585it [08:41,  3.04it/s]

Val loss epoch: 0.0011662682424002924


In [22]:
for idx, batch in tqdm(enumerate(train_loader)):
    if idx==0:

        ids = batch['ids'].to(device, dtype = torch.long)
        mask = batch['mask'].to(device, dtype = torch.long)
        targets = batch['targets'].to(device, dtype = torch.long)

        outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
        loss, tr_logits = outputs.loss, outputs.logits

        flattened_targets = targets.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
    else: break

1it [00:00, 23.26it/s]


In [23]:
tr_logits[0][:100]

tensor([[ 3.4924, -4.1411],
        [-3.5146,  4.0253],
        [-3.6593,  4.0609],
        [ 3.6932, -5.0133],
        [ 4.3382, -5.2632],
        [ 4.7030, -5.1718],
        [ 3.6055, -4.3241],
        [ 4.3836, -4.9923],
        [ 1.4666, -2.1907],
        [-3.6613,  4.5043],
        [-2.8929,  4.0115],
        [ 3.8535, -4.8592],
        [ 4.5035, -5.1777],
        [ 4.3001, -5.1229],
        [ 4.6853, -5.3138],
        [ 3.9623, -4.3422],
        [ 3.9753, -4.3517],
        [ 3.9952, -4.7968],
        [-3.5200,  4.3695],
        [-3.1462,  4.0493],
        [ 3.7984, -5.2290],
        [ 4.3840, -5.1624],
        [ 5.1483, -5.8744],
        [ 4.8928, -5.6284],
        [ 5.1260, -5.6830],
        [ 4.8706, -5.8235],
        [ 5.0296, -5.7687],
        [ 4.9136, -5.5723],
        [ 4.9862, -5.9334],
        [ 3.9249, -4.6148],
        [ 1.4660, -2.1913],
        [ 4.8604, -5.7044],
        [ 4.5439, -5.3385],
        [ 5.3385, -5.9935],
        [ 4.8612, -5.7313],
        [ 5.0252, -6

In [24]:
flattened_targets[:100]

tensor([0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0], device='cuda:0')

In [25]:
targets[0][:100]

tensor([0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0], device='cuda:0')

In [27]:
for i in range(0, 512):
  if targets[0][i] == 1:
      m = tokenizer.decode(ids[0][i])
      print(m)

MT
SS
М
ТС
М
ТС
М
ТС


In [28]:
for i in range(0, 512):
  if flattened_predictions[i] == 1:
      m = tokenizer.decode(ids[0][i])
      print(m)

MT
SS
М
ТС
М
ТС
М
ТС
